# Extraxtion of spikes and LFPs (local field potentials) from the rawdata.



Adjust the directories


In [1]:
scriptdirectory = r"C:\Users\User\Documents\JO\gitkraken\MEA_analysis\CSA_JO"
inputdirectory = r"D:/MEA_DATA_Aachen/PREPROCESSED/20210517_cortex_div11"

outputdirectory = r"D:/MEA_DATA_Aachen/ANALYZED/ID046_analysiert_22102021/analysis_17112021"

## Import Packages

In [2]:
import os
os.chdir(scriptdirectory)

import sys
import numpy as np
import neo
import pandas as pd
import h5py
from hdfviewer.widgets.HDFViewer import HDFViewer
from hdfviewer.widgets.PathSelector import PathSelector
import McsPy
import sys, importlib, os
import McsPy.McsData
import McsPy.McsCMOS
from McsPy import ureg, Q_
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.widgets import Slider
import bokeh.io
import bokeh.plotting
from bokeh.palettes import Spectral11
from scipy.signal import butter, lfilter, freqz, find_peaks, correlate, gaussian, filtfilt
from scipy import stats
from scipy import signal
from scipy import stats
from scipy import signal
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import os
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import McsPy
import McsPy.McsData
from McsPy import ureg, Q_
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from dep_Butterworth_Filter import butter_bandpass, butter_bandpass_filter
import glob
import scipy
import matplotlib.pyplot as plt
from dep_plot_signal_and_spikes import plot_signal_and_spikes_from_bandpassfilteredsignal
import time
import glob
from pathlib import Path



## Many functions are designed before.

These are now imported as one chunk of code snippets.



IMPORTANT: Some of these Functions are simply copied from MCS Py Data Tools Tutorials!


In [3]:

def get_channel_infos(filedirectory, meafile):
    channel_raw_data = McsPy.McsData.RawData(os.path.join(filedirectory, 
                                                          meafile))
    print(channel_raw_data.recordings)
    print(channel_raw_data.comment)
    print(channel_raw_data.date)
    print(channel_raw_data.clr_date)
    print(channel_raw_data.date_in_clr_ticks)
    print(channel_raw_data.file_guid)
    print(channel_raw_data.mea_name)
    print(channel_raw_data.mea_sn)
    print(channel_raw_data.mea_layout)
    print(channel_raw_data.program_name)
    print(channel_raw_data.program_version)
    analognumber = len(channel_raw_data.recordings[0].analog_streams.keys())
    print('In total '+ str(analognumber) 
          + " analog_streams were identified.\n")
    for i in range(len(channel_raw_data.recordings[0].analog_streams.keys())):
        keylist = []
        stream = channel_raw_data.recordings[0].analog_streams[i]
        for key in stream.channel_infos.keys():
                keylist.append(key)
        channel_id = keylist[0]
        datapoints = channel_raw_data.recordings[0].analog_streams[i].channel_data.shape[1]
        samplingfrequency = stream.channel_infos[channel_id].sampling_frequency
        ticks = stream.channel_infos[channel_id].info['Tick']
        time = stream.get_channel_sample_timestamps(channel_id)
        scale_factor_for_second = Q_(1,time[1]).to(ureg.s).magnitude
        time_in_sec = time[0] * scale_factor_for_second
        timelengthrecording_ms = time[0][-1]+ticks
        timelengthrecording_s = (time[0][-1]+ticks)*scale_factor_for_second
        print("analog_stream Nr. " + str(i) + ": ")
        print("datapoints measured = " + str(datapoints))
        print("sampling frequency = " + str(samplingfrequency))
        print("ticks = " + str(ticks))
        print("total recordingtime is: " 
              + str(timelengthrecording_s) + "seconds \n")




def get_MEA_Signal(analog_stream, channel_idx, from_in_s=0, to_in_s=None):
    '''
    Extracts one Channels (channel_idx) Sginal 
    
    :param analog_stream = the analogstream from one recording
    :param channel_idx   = the channel index of the channel where you 
                            extract the values from
    :param from_in_s     = starting point of the range you want to observe 
                            in seconds
    :param to_in_s       = ending point of the range you want to observe. 
                            Default is None (i.e. whole range)
    
    Returns: the signal in uV, time stamps in sec, the sampling frequency
    
    
    '''
    ids = [c.channel_id for c in analog_stream.channel_infos.values()]
    channel_id = ids[channel_idx]
    channel_info = analog_stream.channel_infos[channel_id]
    sampling_frequency = channel_info.sampling_frequency.magnitude

    # get start and end index
    from_idx = max(0, int(from_in_s * sampling_frequency))
    if to_in_s is None:
        to_idx = analog_stream.channel_data.shape[1]
    else:
        to_idx = min(
            analog_stream.channel_data.shape[1], 
            int(to_in_s * sampling_frequency)
            )

    # get the timestamps for each sample
    time = analog_stream.get_channel_sample_timestamps(
        channel_id, from_idx, to_idx
        )

    # scale time to seconds:
    scale_factor_for_second = Q_(1,time[1]).to(ureg.s).magnitude
    time_in_sec = time[0] * scale_factor_for_second

    # get the signal
    signal = analog_stream.get_channel_in_range(channel_id, from_idx, to_idx)

    # scale signal to µV:
    scale_factor_for_uV = Q_(1,signal[1]).to(ureg.uV).magnitude
    signal_in_uV = signal[0] * scale_factor_for_uV
    
    return signal_in_uV, time_in_sec, sampling_frequency, scale_factor_for_second


def get_MEA_Channel_labels(np_analog_for_filter):
    '''
    Gives a List of all MEA Channel Labels (e.g. R12) in the order they appear
    within the recording.
    
    :param analogstream_data = an numpy array shape(channels, data)
    
    '''
    labellist = []
    for i in range(0, len(np_analog_for_filter)):
        #channel_idx = i
        ids = [c.channel_id for c in analog_stream_0.channel_infos.values()]
        channel_id = ids[i]
        channel_info = analog_stream_0.channel_infos[channel_id]
        #print(channel_info.info['Label'])
        labellist.append(channel_info.info['Label'])
    return labellist
    

#@jit(nopython=True)
def detect_threshold_crossings(signal, fs, threshold, dead_time):
    """
    Detect threshold crossings in a signal with dead time and return 
    them as an array

    The signal transitions from a sample above the threshold to a sample 
    below the threshold for a detection and
    the last detection has to be more than dead_time apart 
    from the current one.

    :param signal: The signal as a 1-dimensional numpy array
    :param fs: The sampling frequency in Hz
    :param threshold: The threshold for the signal
    :param dead_time: The dead time in seconds.
    """
    dead_time_idx = dead_time * fs
    threshold_crossings = np.diff(
        (signal <= threshold).astype(int) > 0).nonzero()[0]
    distance_sufficient = np.insert(
        np.diff(threshold_crossings) >= dead_time_idx, 0, True
        )
    while not np.all(distance_sufficient):
        # repeatedly remove all threshold crossings that violate the dead_time
        threshold_crossings = threshold_crossings[distance_sufficient]
        distance_sufficient = np.insert(
            np.diff(threshold_crossings) >= dead_time_idx, 0, True
            )
    return threshold_crossings


def get_next_minimum(signal, index, max_samples_to_search):
    """
    Returns the index of the next minimum in the signal after an index

    :param signal: The signal as a 1-dimensional numpy array
    :param index: The scalar index
    :param max_samples_to_search: The number of samples to search for a 
                                    minimum after the index
    """
    search_end_idx = min(index + max_samples_to_search, signal.shape[0])
    min_idx = np.argmin(signal[index:search_end_idx])
    return index + min_idx


def align_to_minimum(signal, fs, threshold_crossings, search_range, first_time_stamp=0):
    """
    Returns the index of the next negative spike peak for all threshold crossings

    :param signal: The signal as a 1-dimensional numpy array
    :param fs: The sampling frequency in Hz
    :param threshold_crossings: The array of indices where the signal 
                                crossed the detection threshold
    :param search_range: The maximum duration in seconds to search for the 
                         minimum after each crossing
    """
    search_end = int(search_range*fs)
    aligned_spikes = [get_next_minimum(signal, t, search_end) for t in threshold_crossings]
    return np.array(aligned_spikes)


def find_triggers(dset_trigger, tick):
    
    for i in range(0,len(dset_trigger)-1):
        trigger_n=i
        Trigger_An=dset_trigger[trigger_n]
        diff_An=np.diff(Trigger_An)
        peaks, _ = find_peaks(diff_An, height = 2000) #MEA60=0.75
        peaks_off, _ = find_peaks(-diff_An, height = 2000) #""
        if len(peaks)>=0:
            break
    
    if trigger_n ==0:
        odd_peaks= peaks
        odd_peaks_off= peaks_off
    else:
        odd_peaks=peaks
        odd_peaks_off=peaks_off
    #x=np.arange(len(Trigger_An))*tick
    #plt.plot(x, Trigger_An)
    return odd_peaks, odd_peaks_off, diff_An

def spike_on_off(trigger_on, trigger_off, spikedic, tick):
    """
    Takes the dictionary with all spikes and sorts them into either a dictionary for
    spikes while trigger on (=ONdic) or off (=OFFdic)
    
    :param trigger_on =basically created through the find_triggers function 
                        and marks points were stimulation is turned on
    :param trigger_off =see trigger_on but for stimulation off
    :spikedic = dictionary of spikes for each electrode
    :tick
    """
    on=[]
    off=[]
    ONdic ={}
    OFFdic={}
    Trigger_An=[]
    Trigger_Aus=[]
    
    if len(trigger_off)==0:
        Trigger_An=[]
    elif trigger_off[len(trigger_off)-1]>trigger_on[len(trigger_on)-1]:
        Trigger_An=trigger_on*tick
    else:
        Trigger_An=[]
        for n in range(0,len(trigger_on)-1):
            Trigger_An.append(trigger_on[n]*tick)   
        Trigger_An=np.array(Trigger_An)

            
    if len(trigger_on)==0:
        Trigger_Aus=[]
    elif trigger_off[0]>trigger_on[0]:
        Trigger_Aus=trigger_off*tick
    else:
        Trigger_Aus=[]
        for n in range(1,len(trigger_off)):
            Trigger_Aus.append(trigger_off[n]*tick)
        Trigger_Aus=np.array(Trigger_Aus)
    
    Trigger_Aus2=np.insert(Trigger_Aus,0,0)
    
    for key in spikedic:
        ON = []
        OFF = []
        for i in spikedic[key]: #i mit 40 multiplizieren, da Trigger an und aus mit Tick multipliziert sind
            if len(Trigger_An)==0:
                OFF.append(i)
            if any(Trigger_An[foo] < i*tick < Trigger_Aus[foo]  for foo in np.arange(len(Trigger_Aus)-1)):
                ON.append(i)
            elif any(Trigger_Aus2[foo]  < i*tick < Trigger_An[foo]  for foo in np.arange(len(Trigger_An))):
                OFF.append(i)
        ONdic[key]=np.asarray(ON)
        OFFdic[key]=np.asarray(OFF)
    
    return ONdic, OFFdic

#@jit(nopython=True)
def extract_waveforms(signal, fs, spikes_idx, pre, post):
    """
    Extract spike waveforms as signal cutouts around each spike index as a spikes x samples numpy array

    :param signal: The signal as a 1-dimensional numpy array
    :param fs: The sampling frequency in Hz
    :param spikes_idx: The sample index of all spikes as a 1-dim numpy array
    :param pre: The duration of the cutout before the spike in seconds
    :param post: The duration of the cutout after the spike in seconds
    """
    cutouts = []
    pre_idx = int(pre * fs)
    post_idx = int(post * fs)
    for index in spikes_idx:
        if index-pre_idx >= 0 and index+post_idx <= signal.shape[0]:
            cutout = signal[int((index-pre_idx)):int((index+post_idx))]
            cutouts.append(cutout)
    if len(cutouts)>0:
        return np.stack(cutouts)
    
 
#@jit(nopython=True)    
def plot_waveforms(cutouts, fs, pre, post, n=100, color='k', show=True):
    """
    Plot an overlay of spike cutouts

    :param cutouts: A spikes x samples array of cutouts
    :param fs: The sampling frequency in Hz
    :param pre: The duration of the cutout before the spike in seconds
    :param post: The duration of the cutout after the spike in seconds
    :param n: The number of cutouts to plot, or None to plot all. Default: 100
    :param color: The line color as a pyplot line/marker style. Default: 'k'=black
    :param show: Set this to False to disable showing the plot. Default: True
    """
    if n is None:
        n = cutouts.shape[0]
    n = min(n, cutouts.shape[0])
    time_in_us = np.arange(-pre*1000, post*1000, 1e3/fs)
    if show:
        _ = plt.figure(figsize=(12,6))

    for i in range(n):
        _ = plt.plot(time_in_us, cutouts[i,]*1e6, color, linewidth=1, alpha=0.3)
        _ = plt.xlabel('Time (%s)' % ureg.ms)
        _ = plt.ylabel('Voltage (%s)' % ureg.uV)
        _ = plt.title('Cutouts')

    if show:
        plt.show()
        
        
def butter_lowpass_filter(data, cutoff, fs, order):

    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y
        
        
        
def get_next_maximum(signal, index, max_samples_to_search):
    """
    Returns the index of the next maximum in the signal after an index

    :param signal: The signal as a 1-dimensional numpy array
    :param index: The scalar index
    :param max_samples_to_search: The number of samples to search for a 
                                    minimum after the index
    """
    search_end_idx = min(index + max_samples_to_search, signal.shape[0])
    max_idx = np.argmax(signal[index:search_end_idx])
    return index + max_idx



def lfp_crossing_detection(lowpass_filtered_signal, lfp_threshold, minimal_length = 0.05):
    
    '''
    parameters 
    
    lowpass_filtered_signal : array like / list
        the lowpass filtered signal which is considered as the LFP
    
    lfp_threshold : float / int
        the threshold when there is a crossing we regard it as LFP
        deviation
        
    minimal_length : float
        minimal length of a LFP deviation to be considered relevant,
        default is 50ms
        the value is given in seconds
        
    _________
    
    returns:
    
    lfp_down_crossing : list
        list of tuples (a,b) with a start of a deviation, b stop in seconds
        
    lfp_up_crossing : list
        analogue to lfp_down_crossing
        
    amplitudes_down : list
        every down crossing has its local minimum which is the maximal negative amplitude
    
    
    amplitudes_down : list
        every up crossing has its local minimum which is the maximal negative amplitude
    
    '''

    # dicts will have tuples with a start and stop of the lfp crossing
    lfp_up_crossing = []
    lfp_down_crossing = []
    amplitudes_up = []
    amplitudes_down = []
    
    # lfp crosses below threshold
    for i in range(0, len(lowpass_filtered_signal)-1):
        start = 0
        stop = 0
        if (lowpass_filtered_signal[i] < -lfp_threshold) and (lowpass_filtered_signal[i-1] >= -lfp_threshold):
            start = i
            while (lowpass_filtered_signal[i] < -lfp_threshold) and (i < len(lowpass_filtered_signal)-1):
                stop = i
                i += 1
            # filter for at least 50ms  of LFP deviation
            start_seconds = start*scale_factor_for_second*tick + time_in_sec[0] #added since recording do not always start at 0
            stop_seconds = stop*scale_factor_for_second*tick +time_in_sec[0]#added since recording do not always start at 0
            difference_seconds = stop_seconds - start_seconds
            if difference_seconds >= 0.05: # in seconds --> 50 ms
            
                lfp_down_crossing.append((start_seconds, stop_seconds))
                amplitude_point = get_next_minimum(lowpass_filtered_signal, start, stop-start)
                amplitude_down = lowpass_filtered_signal[amplitude_point]
                amplitudes_down.append(amplitude_down)
            
    # lfp crosses above threshold
    
    for i in range(0, len(lowpass_filtered_signal)-1):
        start = 0
        stop = 0
        if (lowpass_filtered_signal[i] > lfp_threshold) and (lowpass_filtered_signal[i-1] <= lfp_threshold):
            start = i
            while (lowpass_filtered_signal[i] > lfp_threshold) and (i < len(lowpass_filtered_signal)-1):
                stop = i
                i += 1
            # filter for at least 50ms  of LFP deviation
            start_seconds = start*scale_factor_for_second*tick +time_in_sec[0]#added since recording do not always start at 0
            stop_seconds = stop*scale_factor_for_second*tick +time_in_sec[0]#added since recording do not always start at 0
            difference_seconds = stop_seconds - start_seconds
            if difference_seconds >= 0.05: # in seconds --> 50 ms
            
                lfp_up_crossing.append((start_seconds, stop_seconds))
                amplitude_point = get_next_maximum(lowpass_filtered_signal, start, stop-start)
                amplitude_up = lowpass_filtered_signal[amplitude_point]
                amplitudes_up.append(amplitude_up)


    return lfp_down_crossing, lfp_up_crossing, amplitudes_down, amplitudes_up


## Loading the data and start to work with it:

We set our low and hightcuts for the filter in Hz, then get a list of all files from our rawdata (.h5 is actually already preprocessed, but will be called rawdata from here on). 

In [4]:

# to save memory:
plt.ioff()


# set filter cuts in Hz
lowcut = 150
highcut = 4500

# Length of cutouts around shapes
pre = 0.001 # 1 ms
post= 0.002 # 2 ms

# divide recording in n seconds long subrecordings
dividing_seconds = 120

# get filelist
os.chdir(inputdirectory)
filelist= glob.glob("*.h5")


resting_spikedic={}
spikedic={}
spikedic_MAD={}
artefactsdic_MAD={}
cutouts_dic ={} 
keylist = []

lfp_ups = {}
lfp_downs = {}
lfp_amplitudes_up = {}
lfp_amplitueds_down = {}

cs_lfp_ups = {}
cs_lfp_downs = {}
cs_lfp_amplitudes_up = {}
cs_lfp_amplitudes_down = {}

lowpass_signal_dic = {}
bandpass_signal_dic = {}
convolved_lowpass_signal_dic = {}


Overview of all files:

In [5]:
filelist

['2021-05-17T11-08-01__cortex_div11_aCSF_ID046_nodrug_spont_1__.h5',
 '2021-05-17T11-28-41__cortex_div11_aCSF_ID046_30µMNorepinephrine_spont_1__.h5',
 '2021-05-17T11-52-18__cortex_div11_aCSF_ID046_reWashaCSF_spont_1__.h5',
 '2021-05-17T11-57-03__cortex_div11_aCSF_ID046_reWashaCSF_spont_2__.h5',
 '2021-05-17T12-21-29__cortex_div11_hCSF_ID046_nodrug_spont_2__.h5']

we concentrate on the recording with 30µM Norepinphrine since it exhibits 
interesting network activity.


In [7]:
file = filelist[1]

In [9]:
# create empty dictionaries / lists to be filled in further analysis
resting_spikedic={}
spikedic={}
cutouts_dic ={} 
keylist = []

filename = file
filedatebase = filename.split('T')[0]
filenamebase = filename.split('__')[1]
filebase = filedatebase + '_' + filenamebase

# print raw channel infos
print('Working on file: ' +filename)
channel_raw_data = McsPy.McsData.RawData(filename)
get_channel_infos(inputdirectory, filename)

Working on file: 2021-05-17T11-28-41__cortex_div11_aCSF_ID046_30µMNorepinephrine_spont_1__.h5
Recording_0 <HDF5 group "/Data/Recording_0" (2 members)>
{0: <Recording label=, AnalogStreams=3, EventStreams=1, duration_time=120400000 microsecond>}

2021-05-17 11:28:41.983040
Montag, 17. Mai 2021
637568477219830367
93cc2871-fddc-4f8a-a10c-710223cf8328
256MEA

MEA_256MEA_MEA2100_252_6
Multi Channel Experimenter
2.15.0.20098
Stream_0 <HDF5 group "/Data/Recording_0/AnalogStream/Stream_0" (3 members)>
ChannelData <HDF5 dataset "ChannelData": shape (252, 3010000), type "<i4">
ChannelDataTimeStamps <HDF5 dataset "ChannelDataTimeStamps": shape (1, 3), type "<i8">
InfoChannel <HDF5 dataset "InfoChannel": shape (252,), type "|V108">
Stream_1 <HDF5 group "/Data/Recording_0/AnalogStream/Stream_1" (3 members)>
ChannelData <HDF5 dataset "ChannelData": shape (1, 3010000), type "<i4">
ChannelDataTimeStamps <HDF5 dataset "ChannelDataTimeStamps": shape (1, 3), type "<i8">
InfoChannel <HDF5 dataset "InfoCha

In [10]:
# import the raw data
analog_stream_0 = channel_raw_data.recordings[0].analog_streams[0]
stream = analog_stream_0
for key in stream.channel_infos.keys():
    keylist.append(key)

Recording_0 <HDF5 group "/Data/Recording_0" (2 members)>
Stream_0 <HDF5 group "/Data/Recording_0/AnalogStream/Stream_0" (3 members)>
ChannelData <HDF5 dataset "ChannelData": shape (252, 3010000), type "<i4">
ChannelDataTimeStamps <HDF5 dataset "ChannelDataTimeStamps": shape (1, 3), type "<i8">
InfoChannel <HDF5 dataset "InfoChannel": shape (252,), type "|V108">
Stream_1 <HDF5 group "/Data/Recording_0/AnalogStream/Stream_1" (3 members)>
ChannelData <HDF5 dataset "ChannelData": shape (1, 3010000), type "<i4">
ChannelDataTimeStamps <HDF5 dataset "ChannelDataTimeStamps": shape (1, 3), type "<i8">
InfoChannel <HDF5 dataset "InfoChannel": shape (1,), type "|V108">
Stream_2 <HDF5 group "/Data/Recording_0/AnalogStream/Stream_2" (3 members)>
ChannelData <HDF5 dataset "ChannelData": shape (1, 3010000), type "<i4">
ChannelDataTimeStamps <HDF5 dataset "ChannelDataTimeStamps": shape (1, 3), type "<i8">
InfoChannel <HDF5 dataset "InfoChannel": shape (1,), type "|V108">


In [11]:
channel_id = keylist[0]
tick = stream.channel_infos[channel_id].info['Tick']
time = stream.get_channel_sample_timestamps(channel_id)
first_recording_timepoint = time[0][0]
scale_factor_for_second = Q_(1,time[1]).to(ureg.s).magnitude
time_in_sec = time[0]*scale_factor_for_second
timelengthrecording_ms = time[0][-1]+tick
timelengthrecording_s = (time[0][-1]+tick)*scale_factor_for_second
fs = int(stream.channel_infos[channel_id].sampling_frequency.magnitude)

In [13]:
analog_stream_0_data = analog_stream_0.channel_data
np_analog_stream_0_data = np.transpose(
    channel_raw_data.recordings[0].analog_streams[0].channel_data
    )

In [14]:
# retranspose the data to make it readable for the filter
np_analog_for_filter = np.transpose(np_analog_stream_0_data)

In [15]:
# delete these streams to save memory
del np_analog_stream_0_data
del analog_stream_0_data
    

In [16]:
# for longer recordings we need to subdivide the signal into snippets 
# 120seconds per snippet was defined above (dividing seconds)
signal_cuts = []
    
starting_point = 0
stopping_point = 0
while starting_point < timelengthrecording_s:
    if starting_point + dividing_seconds >= int(timelengthrecording_s):
        stopping_point = int(timelengthrecording_s)

    else:
        stopping_point =stopping_point + dividing_seconds
    signal_cuts.append((starting_point, stopping_point))

    # set the window one step further:
    starting_point = starting_point + dividing_seconds

In [17]:
signal_cuts

[(0, 120), (120, 120)]

In this example, the second signal cut will only have a few microseconds of data. We will discard this and concentrate on the first 120 seconds.

In [19]:
i = signal_cuts[0]
starting_point = i[0]
stopping_point = i[1]

In [22]:
import time
timestr = time.strftime("%d%m%Y")

# we create an outpath per subdivided recording part

outpath = Path(
    outputdirectory, filebase + '_from_'+str(starting_point) + 
    '_to_' +str(stopping_point) + '_analyzed_on_'+timestr)
try:
    os.mkdir(outpath)
except OSError:
    print ("Creation of the directory %s failed" % outpath)
else:
    print ("Successfully created the directory %s " % outpath)

os.chdir(outpath)

Creation of the directory D:\MEA_DATA_Aachen\ANALYZED\ID046_analysiert_22102021\analysis_17112021\2021-05-17_cortex_div11_aCSF_ID046_30µMNorepinephrine_spont_1_from_0_to_120_analyzed_on_17112021 failed


## We will now go trough the analysis for one channel.

Please note, this is normally done within a for-loop, so that all 252 channels can be analysed at once and saved for further analysis.

In [23]:
i = 0 # to get the first channel

### Spike Detection using the MAD threshold

In [24]:
cutouts_dic ={}
            
channel_idx = i
labellist = get_MEA_Channel_labels(np_analog_for_filter)
signal_in_uV, time_in_sec, sampling_frequency, scale_factor_for_second = get_MEA_Signal(
    analog_stream_0, channel_idx, from_in_s=starting_point,
    to_in_s=stopping_point
    )
bandpassfilteredsignal = butter_bandpass_filter(
    signal_in_uV, lowcut, highcut, sampling_frequency
    )

# This Part is for finding MAD spikes + plotting
noise_mad = np.median(np.absolute(bandpassfilteredsignal)) / 0.6745
threshold = -5* noise_mad
artefact_threshold = -8* noise_mad
crossings = detect_threshold_crossings(
    bandpassfilteredsignal, sampling_frequency, 
    threshold, dead_time=0.001
    )
spikes=align_to_minimum(
    bandpassfilteredsignal, fs, crossings, search_range=0.003, 
    first_time_stamp=first_recording_timepoint
    )

if len(spikes) < 10:
    artefact_crossings = detect_threshold_crossings(
        bandpassfilteredsignal, sampling_frequency, 
        artefact_threshold, dead_time=0.001
        )
    artefacts = align_to_minimum(
        bandpassfilteredsignal, fs, artefact_crossings, search_range=0.003, 
        first_time_stamp=first_recording_timepoint
        )


# this line accoutns for a starting point of the recording that is != 0
spikes = spikes + int(time_in_sec[0]/(scale_factor_for_second*tick)) 
channellabel = labellist[i]
spikedic_MAD[channellabel] = spikes
bandpass_signal_dic[channellabel] = bandpassfilteredsignal
#artefactsdic_MAD[channellabel] = artefacts
print('iteration ' + str(i) + ' channel: ' +str(channellabel))


iteration 0 channel: G13


In [27]:
#check if any spikes were detected:
len(spikes)

24

### Next: Crossings of the Local Field Potential (LFP)

In [28]:
T = timelengthrecording_s         # Sample Period
fs = fs      # sample rate, Hz
cutoff = 100  # desired cutoff frequency of the filter of lowpass
nyq = 0.5 * fs  # Nyquist Frequency
order = 2       # sin wave can be approx represented as quadratic
n = int(T * fs) # total number of samples

butter_lowpass_filtered_signal = butter_lowpass_filter(signal_in_uV, cutoff, fs, order)


lfp_std = np.std(butter_lowpass_filtered_signal)
lfp_mean = np.mean(butter_lowpass_filtered_signal)
threshold_LFP = 3*lfp_std


down_cross, up_cross, amp_down, amp_up = lfp_crossing_detection(
    butter_lowpass_filtered_signal, threshold_LFP, minimal_length=0.03)


# this is also done for a convolved, i.e., smoothed version of the signal
#Modulation for the LFP Start

convolved_signal = np.convolve(butter_lowpass_filtered_signal, np.ones(3000)/3000, mode='full')


length_cutter = len(butter_lowpass_filtered_signal)

cs = convolved_signal[:length_cutter]

cs_threshold = np.std(cs)*2

cs_down_cross, cs_up_cross, cs_amp_down, cs_amp_up = lfp_crossing_detection(
    convolved_signal, cs_threshold, minimal_length=0.01)


In [29]:
# we need to reuse most of the data just calculated
# therefore, they are stored in a dictionary with the channellabel
lfp_ups[channellabel] = up_cross
lfp_downs[channellabel] = down_cross
lfp_amplitudes_up[channellabel] = amp_up
lfp_amplitueds_down[channellabel] = amp_down

cs_lfp_ups[channellabel] = cs_up_cross
cs_lfp_downs[channellabel] = cs_down_cross
cs_lfp_amplitudes_up[channellabel] = cs_amp_up
cs_lfp_amplitudes_down[channellabel] = cs_amp_down

lowpass_signal_dic[channellabel] = butter_lowpass_filtered_signal
convolved_lowpass_signal_dic[channellabel] = convolved_signal

In [30]:
# let us look at one example:
lfp_ups

{'G13': [(90.56384, 90.90379999999999),
  (106.78151999999999, 107.05624),
  (107.05764, 107.14112)]}

so the channel G13 has upcrossings of the LFP at three different times

In [32]:
if len(spikes) > 3:

    #only extract cutouts when they are relevant
    # extracting the cutouts takes a lot of working memory
    cutouts = extract_waveforms(
            bandpassfilteredsignal, sampling_frequency, spikes, 
            pre, post
            )
    cutouts_dic[channellabel] = cutouts


    plt.style.use("seaborn-white")


    # figure 1: signal with threshold
    fig1, ax = plt.subplots(1, 1, figsize=(20, 10))
    ax = plt.plot(time_in_sec, bandpassfilteredsignal, c="#45858C")
    ax = plt.plot([time_in_sec[0], time_in_sec[-1]], [threshold, threshold], c="#297373")
    ax = plt.plot(spikes*tick*scale_factor_for_second, [threshold-1]*(spikes*tick*scale_factor_for_second).shape[0], 'ro', ms=2, c="#D9580D")
    ax = plt.title('Channel %s' %channellabel)
    ax = plt.xlabel('Time in Sec, Threshold: %s' %threshold)
    ax = plt.ylabel('µ volt')
    ax = plt.plot(time_in_sec, butter_lowpass_filtered_signal, c='#F29829', linewidth=0.5)
    for i in down_cross:
        ax = plt.axvspan(i[0], i[1], color='#5D7CA6', alpha=0.2)
    for i in up_cross:
        ax = plt.axvspan(i[0], i[1], color='#BF214B', alpha=0.2)
    fig_1_name = filebase+'_signal_'+channellabel+'MAD_THRESHOLD.png'
    if not os.path.exists(outpath):
        os.mkdir(outpath) 
    fullfig_1_name = Path(outpath, fig_1_name)
    fig1.savefig(fullfig_1_name)
    plt.close(fig1) 
    plt.clf()




In [33]:

#figure 2: waveforms 
fig2, ax2 = plt.subplots(1, 1, figsize=(12,6))
#ax2 is a plot of the waveform cutouts
n = 100
n = min(n, cutouts.shape[0])
time_in_us = np.arange(-pre*1000, post*1000, 1e3/fs)
cutout_mean = np.mean(cutouts, axis=0)
for i in range(n):
    ax2 = plt.plot(time_in_us, cutouts[i,]*1e6, color='black', linewidth=1, alpha=0.3)
    ax2 = plt.plot(time_in_us, cutout_mean*1e6, color="red", linewidth=1, alpha=0.3)
    ax2 = plt.xlabel('Time (%s)' % ureg.ms)
    ax2 = plt.ylabel('Voltage (%s)' % ureg.uV)
    ax2 = plt.title('Cutouts of Channel %s' %channellabel)

fig_2_name = filebase+'_waveforms_'+channellabel+'MAD_THRESHOLD.png'
if not os.path.exists(outpath):
    os.mkdir(outpath) 
fullfig_2_name = Path(outpath, fig_2_name)
fig2.savefig(fullfig_2_name)
plt.close(fig2) 
plt.clf()

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\MEA_DATA_Aachen\\ANALYZED\\ID046_analysiert_22102021\\analysis_17112021\\2021-05-17_cortex_div11_aCSF_ID046_30µMNorepinephrine_spont_1_from_0_to_120_analyzed_on_17112021\\2021-05-17_cortex_div11_aCSF_ID046_30µMNorepinephrine_spont_1_waveforms_G13MAD_THRESHOLD.png'